### Authenticate
Get the client id and secret from env vars for prod.
In dev we can create a new client each time

In [ ]:
import { registerSystem } from './helpers/gqlHandlers.ts'
import { authenticate, getTokenForSystemClient } from './helpers/authentication.ts'
import { CLIENT_ID, CLIENT_SECRET } from './helpers/vars.ts'
import { ADMIN_USERNAME, ADMIN_PASSWORD } from './helpers/vars.ts'


let clientId: string | null = null
let clientSecret: string | null = null

if (CLIENT_ID && CLIENT_SECRET) {
  clientId = CLIENT_ID
  clientSecret = CLIENT_SECRET
} else {
  // For dev mode
  const adminToken = await authenticate(ADMIN_USERNAME, ADMIN_PASSWORD)
  const systemRegistration = await registerSystem(adminToken)

  clientId = systemRegistration.data.registerSystem.system.clientId
  clientSecret = systemRegistration.data.registerSystem.clientSecret
}

const sysToken = await getTokenForSystemClient(clientId, clientSecret)

### Sync locations

On update to v1.9, the postgres db won't have locations so this is required to sync them from mongo

In [ ]:
import { syncLocations } from './helpers/gqlHandlers.ts'

const res = await syncLocations(sysToken)
res

### Fetch birth and death events

In [ ]:
import { fetchEvents } from './helpers/formsHandlers.ts'
import { extractFieldType } from './helpers/utils.ts'

const events = await fetchEvents(sysToken)

const ignoredFields = ['DIVIDER', 'PARAGRAPH']

const birthEvent = events.find((x) => x.id === 'birth')
const deathEvent = events.find((x) => x.id === 'death')

const birthEventFields = new Set(
  extractFieldType(birthEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)

const deathEventFields = new Set(
  extractFieldType(deathEvent, 'fields')
    .filter((x) => !ignoredFields.includes(x.type))
    .map((f) => f.id)
    .filter((x) => x)
)


### Get all potential resolvers
Use only resolvers for used event fields to avoid nulls

In [ ]:
import defaultResolvers, {
  defaultBirthResolver,
  defaultDeathResolver,
} from './helpers/defaultResolvers.ts'
import { countryResolver } from './countryData/countryResolvers.ts'

const allResolvers = { ...defaultResolvers, ...countryResolver }

const birthResolver = Object.fromEntries(
  Object.entries({...defaultBirthResolver, ...allResolvers}).filter(([key, _value]) =>
    [...birthEventFields].includes(key)
  )
)

const deathResolver = Object.fromEntries(
  Object.entries({...defaultDeathResolver, ...allResolvers}).filter(([key, _value]) =>
    [...deathEventFields].includes(key)
  )
)

### Migrate births

In [ ]:
import { EVENT } from './helpers/vars.ts'
import {
  fetchBirthRegistration,
  bulkImport,
  fetchAllBirthRegistrations
} from './helpers/gqlHandlers.ts'
import { transform } from './helpers/transform.ts'
import { batch, getIndexErrors } from './helpers/utils.ts'
import { RECORD_SKIP, SINGLE_RECORD } from './helpers/vars.ts'

const individualFailures = []

const migrateBirth = async (entryIds) => {
  const transformed = []

  for (const entryId of entryIds) {
    // Get entry
    const birthRegistrationData = await fetchBirthRegistration(
      entryId,
      sysToken
    )
    if (!birthRegistrationData.data.fetchBirthRegistration) {
      console.error(JSON.stringify(birthRegistrationData, null, 2))
      throw new Error(`No data for ${entryId}`)
    }

    let document
    try {
      document = transform(
        birthRegistrationData.data.fetchBirthRegistration,
        birthResolver,
        'birth'
      )
      transformed.push(document)
    } catch (err) {
      // TODO remove for PROD
      console.log('document')
      console.log(JSON.stringify(document, null, 2))
      console.log('birthRegistrationData')
      console.log(JSON.stringify(birthRegistrationData))
      individualFailures.push(entryId)
    }
  }

  return await bulkImport(transformed, sysToken)
}

if (EVENT === 'birth' && !SINGLE_RECORD) {
  const skippedPages = RECORD_SKIP ? Math.floor(RECORD_SKIP / 1000) : 0
  const pageSize = 1000
  const batchSize = 100
  let itemsRemaining = 0
  let page = 1 + skippedPages
  let totalProcessed = skippedPages * pageSize

  if (RECORD_SKIP) {
    console.log(`Skipping first ${skippedPages * pageSize} records`)
  }
  do {
    const birthRegistrations = await fetchAllBirthRegistrations(
      sysToken,
      page,
      pageSize
    )
    if (!birthRegistrations.data.searchEvents) {
      console.error(JSON.stringify(birthRegistrations, null, 2))
      throw new Error('No data from searchEvents')
    }

    const { results, totalItems } = birthRegistrations.data.searchEvents
    const birthIds = results.map((x) => x.id)

    console.log(
      `Processing next page of ${birthIds.length} of ${totalItems} total records`
    )

    const batches = batch(birthIds, batchSize)

    for (const batch of batches) {
      const indexResult = await migrateBirth(batch)
      console.log(`  - Processed batch of ${batch.length} records`)
      const errors = getIndexErrors(indexResult)
      if (errors) {
        console.error('Errors during bulk import', errors)
        throw new Error(errors)
      }
    }

    totalProcessed += birthIds.length
    itemsRemaining = Math.max(0, totalItems - totalProcessed)

    console.log(
      `Processed ${totalProcessed} of ${totalItems} birth registrations... with ${itemsRemaining} remaining.`
    )
    page += 1
  } while (itemsRemaining > 0)
} else if (EVENT === 'birth' && SINGLE_RECORD) {
  const birthRegistrationData = await fetchBirthRegistration(SINGLE_RECORD, sysToken)
  if (!birthRegistrationData.data.fetchBirthRegistration) {
    console.error(JSON.stringify(birthRegistrationData, null, 2))
    throw new Error(`No data for ${SINGLE_RECORD}`)
  }
  
  const indexResult = await migrateBirth([SINGLE_RECORD])
  const errors = getIndexErrors(indexResult)
  if (errors) {
    console.error('Errors during bulk import', errors)
    throw new Error(errors)
  }
}


### Migrate Deaths

In [ ]:
import {
  fetchDeathRegistration,
  fetchAllDeathRegistrations,
} from './helpers/gqlHandlers.ts'

const migrateDeath = async (entryIds) => {
  const transformed = []

  for (const entryId of entryIds) {
    // Get entry
    const deathRegistrationData = await fetchDeathRegistration(
      entryId,
      sysToken
    )
    if (!deathRegistrationData.data.fetchDeathRegistration) {
      console.error(JSON.stringify(deathRegistrationData, null, 2))
      throw new Error(`No data for ${entryId}`)
    }

    let document
    try {
      document = transform(
        deathRegistrationData.data.fetchDeathRegistration,
        deathResolver,
        'death'
      )
      transformed.push(document)
    } catch (err) {
      // TODO remove for PROD
      console.log('document')
      console.log(JSON.stringify(document, null, 2))
      console.log('deathRegistrationData')
      console.log(JSON.stringify(deathRegistrationData))
      individualFailures.push(entryId)
    }
  }

  return await bulkImport(transformed, sysToken)
}

if (EVENT === 'death' && !SINGLE_RECORD) {
  const skippedPages = RECORD_SKIP ? Math.floor(RECORD_SKIP / 1000) : 0
  const pageSize = 1000
  const batchSize = 100
  let itemsRemaining = 0
  let page = 1 + skippedPages
  let totalProcessed = skippedPages * pageSize

  if (RECORD_SKIP) {
    console.log(`Skipping first ${skippedPages * pageSize} records`)
  }

  do {
    const deathRegistrations = await fetchAllDeathRegistrations(
      sysToken,
      page,
      pageSize
    )
    if (!deathRegistrations.data.searchEvents) {
      console.error(JSON.stringify(deathRegistrations, null, 2))
      throw new Error('No data from searchEvents')
    }
    const { results, totalItems } = deathRegistrations.data.searchEvents
    const deathIds = results.map((x) => x.id)
    console.log(
      `Processing next page of ${deathIds.length} of ${totalItems} total records`
    )

    const batches = batch(deathIds, batchSize)

    for (const batch of batches) {
      const indexResult = await migrateDeath(batch)
      console.log(`  - Processed batch of ${batch.length} records`)
      const errors = getIndexErrors(indexResult)
      if (errors) {
        console.error('Errors during bulk import', errors)
        throw new Error(errors)
      }
    }

    totalProcessed += deathIds.length
    itemsRemaining = Math.max(0, totalItems - totalProcessed)

    console.log(
      `Processed ${totalProcessed} of ${totalItems} death registrations... with ${itemsRemaining} remaining.`
    )
    page += 1
  } while (itemsRemaining > 0)
} else if (EVENT === 'death' && SINGLE_RECORD) {
  const deathRegistrationData = await fetchDeathRegistration(SINGLE_RECORD, sysToken)
  if (!deathRegistrationData.data.fetchDeathRegistration) {
    console.error(JSON.stringify(deathRegistrationData, null, 2))
    throw new Error(`No data for ${SINGLE_RECORD}`)
  }
  
  const indexResult = await migrateDeath([SINGLE_RECORD])
  const errors = getIndexErrors(indexResult)
  if (errors) {
    console.error('Errors during bulk import', errors)
    throw new Error(errors)
  }
}


In [ ]:
console.log('Migration completed.')
if (individualFailures.length > 0) {
  console.log(
    `However, there were ${individualFailures.length} individual records that failed to migrate:`
  )
  console.log(individualFailures)
} else {
  console.log('All individual records migrated successfully.')
}

In [ ]:
import { reindex } from "./helpers/gqlHandlers.ts";

const reindexResponse = await reindex(sysToken);
reindexResponse


### Output results


In [ ]:
console.log('🍞 Declarations succesfully migrated:')
